## Estudo de critérios de transição de sigmoides para modelagem matemática de múltiplas ondas epidemiológicas

Testando diferentes filtros

Importando bibliotecas e módulos implementados

In [ ]:
# External libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import butter, lfilter

# Implemented modules
import optimize
# Obs: modified version of the new_wave module
import new_wave_filt
from new_wave_filt import get_transition_points

Carregando dados

In [ ]:
# Import data
data = pd.read_csv("../Datasets/osaka.csv") 
city_name = 'Osaka' 
indicator='cases'

In [ ]:
if(indicator == 'deaths'):
    #deaths
    acc_data = data.cumulative_deceased
else:
    # cases
    #acc_data = data.total_confirmed
    acc_data = data.cumulative_confirmed #(Japan)

normalized_acc_data = acc_data / max(acc_data)
t = np.linspace(0, len(acc_data)-1, len(acc_data))

normalized_acc_data = normalized_acc_data.tolist()

if(indicator == 'deaths'):
    daily_data = data.new_deceased
else:
    daily_data = data.new_confirmed


scaling_factor = 500
acc_data = acc_data / scaling_factor
daily_data = list(daily_data/ scaling_factor)

Definindo condições iniciais de cada sigmoide no modelo

In [ ]:
# Initial Conditions

def initial_cond_0(y_t):
    #A0 = 100
    A0 = 2*max(y_t)
    tp0 = (2/3)*len(y_t)
    delta0 = (1/4)*len(y_t)
    nu0 = 1
    return [A0, tp0, delta0, nu0]

optimize.initial_cond = initial_cond_0

def update_cond_nw(A0, tp0):
    tp0 += 150
    A0 *= 0.005
    return [A0, tp0]
optimize.update_cond = update_cond_nw

## Testando diferentes filtros

In [ ]:
# Moving average filter
def moving_average(x, win_size):
    filtered = np.convolve(x, np.ones(win_size), 'valid') / win_size
    filtered = np.append(np.zeros(win_size-1), filtered) # fill the w-1 first slots with zeros
    return filtered

# Median filter
def median_filter(x, win_size):
    S = 1
    nrows = ((x.size-win_size)//S)+1
    n = x.strides[0]
    strided = np.lib.stride_tricks.as_strided(x, shape=(nrows,win_size), strides=(S*n,n))
    filtered  = np.median(strided,axis=1)
    filtered = np.append(np.zeros(win_size-1), filtered) # fill the w-1 first slots with zeros
    return filtered

def butterworth_lowpass_filter(data, cutoff_freq, fs, order=2):
    #fs is the sampling rate
    nyq = 0.5 * fs
    normal_cutoff = cutoff_freq / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    # lfilter apply filter along one dimension
    y = lfilter(b, a, data)
    return y

### 1 - Filtros padrão

In [ ]:
def filter_data(data):    

    plt.figure(figsize=(12, 6))
    plt.title(f"Filtering process - {city_name}")
    plt.xlabel("t (days)")
    plt.ylabel("cases / day")

    # Plotting unfiltered version
    plt.plot(data, label="unfiltered", c='silver', linestyle='dashed')

    # Moving average with 14-day window
    filtered_data = moving_average(data, 14)
    plt.plot(filtered_data, label="filtered (ma-14)")
    
    # 2nd Order Low-Pass Filter with 14-day window
    order = 2
    fs = len(data) # sampling rate       
    cutoff = 14 # cutoff freq.
    filtered_data =  butterworth_lowpass_filter(filtered_data, cutoff, fs, order)
    plt.plot(filtered_data, label="filtered (ma-14 + bw-low-pass)")

    # Median filter with 14-day window
    filtered_data = median_filter(filtered_data, 14)
    plt.plot(filtered_data, label="filtered (ma-14 + bw-low-pass + median-14)")

    # Reduce the delay effect introduced by the filtering process
    # Advance the signal by 25 days
    n_days_shift = 25
    filtered_data = filtered_data[n_days_shift:]    
    plt.plot(filtered_data, label=f"shifted signal ({n_days_shift} days)")

    plt.legend()
    plt.show()

    return filtered_data

new_wave_filt.filter_data = filter_data

# =================================================================================================

# Transition Points
x_nw = get_transition_points(acc_data, visual=True, threshold=2e-6, indicator = indicator)
x_nw = x_nw[1:6]

print('x_nw:', x_nw)

sig_params = optimize.fit_data(acc_data, 
                               daily_data, 
                               city_name, 
                               x_nw, 
                               indicator = indicator, 
                               n_weeks_pred = 0,
                               scaling_factor = scaling_factor
                            )

In [ ]:
sig_params

### 2 - Só média móvel

In [ ]:
def filter_data(data):    

    plt.figure(figsize=(12, 6))
    plt.title(f"Filtering process - {city_name}")
    plt.xlabel("t (days)")
    plt.ylabel("cases / day")

    # Plotting unfiltered version
    plt.plot(data, label="unfiltered", c='silver', linestyle='dashed')

    # Moving average with 21-day window
    filtered_data = moving_average(data, 21)
    plt.plot(filtered_data, label="filtered (ma-21)")

    plt.legend()
    plt.show()

    return filtered_data

new_wave_filt.filter_data = filter_data

# =================================================================================================

# Transition Points
x_nw = get_transition_points(acc_data, visual=True, threshold=2e-6, indicator = indicator)
x_nw = x_nw[1:6]

print('x_nw:', x_nw)

sig_params = optimize.fit_data(acc_data, 
                               daily_data, 
                               city_name, 
                               x_nw, 
                               indicator = indicator, 
                               n_weeks_pred = 0,
                               scaling_factor = scaling_factor
                            )

### 3 - Só filtro butterworth + avanço do sinal

In [ ]:
def filter_data(data):    

    plt.figure(figsize=(12, 6))
    plt.title(f"Filtering process - {city_name}")
    plt.xlabel("t (days)")
    plt.ylabel("cases / day")

    # Plotting unfiltered version
    plt.plot(data, label="unfiltered", c='silver', linestyle='dashed')

    # 2nd Order Low-Pass Filter with 14-day window
    order = 2
    fs = len(data) # sampling rate       
    cutoff = 14 # cutoff freq.
    filtered_data =  butterworth_lowpass_filter(data, cutoff, fs, order)
    plt.plot(filtered_data, label="filtered (bw-low-pass)")

    # Reduce the delay effect introduced by the filtering process
    # Advance the signal by 15 days
    n_days_shift = 15
    filtered_data = filtered_data[n_days_shift:]    
    plt.plot(filtered_data, label=f"shifted signal ({n_days_shift} days)")

    plt.legend()
    plt.show()

    return filtered_data

new_wave_filt.filter_data = filter_data

# =================================================================================================

# Transition Points
x_nw = get_transition_points(acc_data, visual=True, threshold=2e-6, indicator = indicator)
x_nw = x_nw[1:6]

print('x_nw:', x_nw)

sig_params = optimize.fit_data(acc_data, 
                               daily_data, 
                               city_name, 
                               x_nw, 
                               indicator = indicator, 
                               n_weeks_pred = 0,
                               scaling_factor = scaling_factor
                            )

### 4 - Só mediana + avanço

In [ ]:
def filter_data(data):    

    plt.figure(figsize=(12, 6))
    plt.title(f"Filtering process - {city_name}")
    plt.xlabel("t (days)")
    plt.ylabel("cases / day")

    # Plotting unfiltered version
    plt.plot(data, label="unfiltered", c='silver', linestyle='dashed')

    # Median filter with 14-day window
    filtered_data = median_filter(data, 14)
    plt.plot(filtered_data, label="filtered (ma-14 + bw-low-pass + median-14)")

    # Reduce the delay effect introduced by the filtering process
    # Advance the signal by 5 days
    n_days_shift = 5
    filtered_data = filtered_data[n_days_shift:]    
    plt.plot(filtered_data, label=f"shifted signal ({n_days_shift} days)")

    plt.legend()
    plt.show()

    return filtered_data

new_wave_filt.filter_data = filter_data

# =================================================================================================

# Transition Points
x_nw = get_transition_points(acc_data, visual=True, threshold=2e-6, indicator = indicator)
x_nw = x_nw[1:6]

print('x_nw:', x_nw)

sig_params = optimize.fit_data(acc_data, 
                               daily_data, 
                               city_name, 
                               x_nw, 
                               indicator = indicator, 
                               n_weeks_pred = 0,
                               scaling_factor = scaling_factor
                            )

### 5 - Média móvel + Mediana + avanço

In [ ]:
def filter_data(data):    

    plt.figure(figsize=(12, 6))
    plt.title(f"Filtering process - {city_name}")
    plt.xlabel("t (days)")
    plt.ylabel("cases / day")

    # Plotting unfiltered version
    plt.plot(data, label="unfiltered", c='silver', linestyle='dashed')

    # Moving average with 14-day window
    filtered_data = moving_average(data, 14)
    plt.plot(filtered_data, label="filtered (ma-14)")
    # Median filter with 14-day window
    filtered_data = median_filter(filtered_data, 14)
    plt.plot(filtered_data, label="filtered (ma-14 + median-14)")

    # Reduce the delay effect introduced by the filtering process
    # Advance the signal by 25 days
    n_days_shift = 10
    filtered_data = filtered_data[n_days_shift:]    
    plt.plot(filtered_data, label=f"shifted signal ({n_days_shift} days)")

    plt.legend()
    plt.show()

    return filtered_data

new_wave_filt.filter_data = filter_data

# =================================================================================================

# Transition Points
x_nw = get_transition_points(acc_data, visual=True, threshold=2e-6, indicator = indicator)
x_nw = x_nw[1:6]

print('x_nw:', x_nw)

sig_params = optimize.fit_data(acc_data, 
                               daily_data, 
                               city_name, 
                               x_nw, 
                               indicator = indicator, 
                               n_weeks_pred = 0,
                               scaling_factor = scaling_factor
                            )

### Bônus - Experimentando frequência de corte do Butterworth
Teste antigo que fiz, mas agora com dados do Japão

In [ ]:
plt.figure(figsize=(12,6))
plt.title("Experimenting with Butterworth filter's cutoff frequency")
plt.xlabel("t (days)")
plt.ylabel("cases / day")

unf_daily_n_cases = daily_data

plt.plot(unf_daily_n_cases, label="unfiltered", c='silver', linestyle='dashed')

order = 2
fs = len(data) # sampling rate       

cutoff = 2 # cutoff freq.
filtered =  butterworth_lowpass_filter(unf_daily_n_cases, cutoff, fs, order)
plt.plot(filtered, label=f"cutoff = {cutoff}")

cutoff = 7 # cutoff freq.
filtered =  butterworth_lowpass_filter(unf_daily_n_cases, cutoff, fs, order)
plt.plot(filtered, label=f"cutoff = {cutoff}")

cutoff = 14 # cutoff freq.
filtered =  butterworth_lowpass_filter(unf_daily_n_cases, cutoff, fs, order)
plt.plot(filtered, label=f"cutoff = {cutoff}")

cutoff = 30 # cutoff freq.
filtered =  butterworth_lowpass_filter(unf_daily_n_cases, cutoff, fs, order)
plt.plot(filtered, label=f"cutoff = {cutoff}")

plt.legend()
plt.show()
